In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%cd /content/drive/MyDrive/Courses-UMICH/EECS 583/representation/
%pwd

/content/drive/MyDrive/Courses-UMICH/EECS 583/representation


'/content/drive/MyDrive/Courses-UMICH/EECS 583/representation'

In [20]:
import json

def load_json_data(filename):
  with open(filename) as f:
    data_str = f.read()
    # print(data_str)
    data = json.loads(data_str)
  return data

cdfg_data = load_json_data('cdfg.json')
programl_data = load_json_data('prograML.json')

edge = [start, type, end]  
nodes: list of node types

In [21]:
print(len(cdfg_data))
print(len(programl_data))

679
679


In [22]:
SEED=42
from sklearn.model_selection import train_test_split

# train/test/valid_cdfg_data, 0.4/0.2/0.4
tr_va_cdfg_data, test_cdfg_data = train_test_split(cdfg_data, test_size=0.2, random_state=SEED)
train_cdfg_data, valid_cdfg_data =  train_test_split(tr_va_cdfg_data, test_size=0.5, random_state=SEED)

# train/test/valid_programl_data, 0.4/0.2/0.4
tr_va_programl_data, test_programl_data = train_test_split(programl_data, test_size=0.2, random_state=SEED)
train_programl_data, valid_programl_data =  train_test_split(tr_va_programl_data, test_size=0.5, random_state=SEED)

In [23]:
print(cdfg_data[:5])
print(programl_data[:5])

[{'y': 1, 'x': {'aux_in': [2048, 255], 'code_rep': {'nodes': [0, 1, 1, 1, 1, 2, 2, 3, 2, 2, 2, 2, 4, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 2, 9, 10, 9, 2, 11, 12, 13, 14, 9, 9, 9, 12, 12, 8, 8, 12, 8, 9, 9, 15, 16, 2, 13, 14, 9, 16, 2, 6, 7, 17, 18, 19, 20, 21, 19, 6, 7, 7, 7, 20, 22, 20, 3, 23, 19, 20, 3, 21, 3, 9, 2, 4, 20, 21, 19, 20, 2, 9, 2, 20, 19, 20, 7, 4, 20], 'edges': [[0, 1, 28], [0, 1, 55], [0, 1, 64], [1, 1, 20], [2, 1, 86], [3, 1, 52], [3, 1, 60], [3, 1, 66], [4, 1, 61], [4, 1, 62], [5, 2, 6], [5, 0, 18], [5, 1, 18], [5, 1, 51], [5, 1, 57], [6, 2, 7], [6, 2, 8], [6, 0, 19], [6, 1, 19], [7, 0, 21], [7, 1, 21], [7, 1, 22], [7, 1, 24], [7, 1, 26], [8, 2, 9], [8, 0, 33], [8, 1, 33], [9, 2, 10], [9, 0, 35], [9, 1, 35], [9, 1, 37], [10, 2, 11], [10, 0, 36], [10, 1, 36], [10, 1, 38], [11, 2, 12], [11, 0, 49], [11, 1, 50], [12, 2, 13], [12, 0, 13], [13, 0, 55], [14, 2, 15], [14, 2, 16], [14, 2, 17], [14, 0, 77], [15, 0, 16], [15, 1, 82], [16, 0, 81], [16, 1, 81], [17, 0, 83], [18, 0, 6],

# Generate unpadded data

In [28]:
import json
from collections import defaultdict

def generate_input_data(data, filename):
  max_nodes_size = 0
  max_feat_val = 0

  for d in data:
    info = {}

    nodes_feats = d['x']['code_rep']['nodes'] # list
    edges = d['x']['code_rep']['edges'] # list of lists

    num_nodes = len(nodes_feats)
    num_edges = len(edges)

    max_nodes_size = max(num_nodes, max_nodes_size)

    info['seq'] = str(d['y'])
    info['g_ids'] = {}
    info['g_ids_features'] = {}
    info['g_adj'] = {}

    for i in range(num_nodes):
      info['g_ids'][str(i)] = i
      info['g_ids_features'][str(i)] = str(nodes_feats[i]) # TODO: change here to add edge features
      max_feat_val = max(max_feat_val, nodes_feats[i])
    
    adj_lst = defaultdict(list) # string -> list of ints
    for e in edges:
      v1, e_type, v2 = e[0], e[1], e[2]
      adj_lst[str(v1)].append(v2)
      adj_lst[str(v2)].append(v1) # could be added multiple times

    # Set default values for nodes w/o adj nodes
    for v in range(num_nodes):
      if str(v) in adj_lst:
        continue
      adj_lst[str(v)] = []

    info['g_adj'] = adj_lst

    assert len(info['g_ids']) == len(info['g_ids_features'])
    assert len(info['g_ids']) == len(info['g_adj'])

    # with open(filename, "a") as f: # append
    #   json.dump(info, f)
    #   f.write('\n')

  print("Max num nodes in this split:", max_nodes_size)
  print("Max feat val in this split:", max_feat_val)

In [30]:
data_dir = "../Colab Notebooks/Graph2Seq/data/devmap/"

cdfg_data_dict = {
    'train': train_cdfg_data,
    'dev': valid_cdfg_data,
    'test': test_cdfg_data,
}

programl_data_dict = {
    'train': train_programl_data,
    'dev': valid_programl_data,
    'test': test_programl_data,
}

for data_type in ["cdfg", "programl"]:
  for split in ['train', 'dev', 'test']:
    if data_type == "cdfg":
      filepath = data_dir + 'cdfg_' + split + '.data'
      data_dict = cdfg_data_dict
      generate_input_data(data_dict[split], filepath)
    else:
      filepath = data_dir + 'programl_' + split + '.data' 
      data_dict = programl_data_dict
      generate_input_data(data_dict[split], filepath)
    print(f"Generated data at {filepath}")

Max num nodes in this split: 3106
Max feat val in this split: 69
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/cdfg_train.data
Max num nodes in this split: 1932
Max feat val in this split: 69
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/cdfg_dev.data
Max num nodes in this split: 1134
Max feat val in this split: 68
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/cdfg_test.data
Max num nodes in this split: 6216
Max feat val in this split: 158
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/programl_train.data
Max num nodes in this split: 4010
Max feat val in this split: 158
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/programl_dev.data
Max num nodes in this split: 3045
Max feat val in this split: 159
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/programl_test.data


# Generate padded data

In [31]:
MAX_CDFG_NODES = 3106
MAX_CDFG_FEAT = 69

MAX_PROGRAML_NODES = 6216
MAX_PROGRAML_FEAT = 159

In [37]:
# Pad nodes to max size
import json
from collections import defaultdict

def generate_padded_input_data(data, filename, max_len, max_feat):
  for d in data:
    info = {}

    nodes_feats = d['x']['code_rep']['nodes'] # list
    edges = d['x']['code_rep']['edges'] # list of lists

    num_nodes = len(nodes_feats)
    num_edges = len(edges)

    info['seq'] = str(d['y'])
    info['g_ids'] = {}
    info['g_ids_features'] = {}
    info['g_adj'] = {}

    for i in range(num_nodes):
      info['g_ids'][str(i)] = i
      info['g_ids_features'][str(i)] = str(nodes_feats[i]) # TODO: change here to add more features (e.g. edge features)
    
    adj_lst = defaultdict(list) # string -> list of ints
    for e in edges:
      v1, e_type, v2 = e[0], e[1], e[2]
      adj_lst[str(v1)].append(v2)
      adj_lst[str(v2)].append(v1) # could be added multiple times

    info['g_adj'] = adj_lst

    # padding
    for i in range(num_nodes, max_len):
      info['g_ids'][str(i)] = i
      info['g_ids_features'][str(i)] = str(max_feat + 1) # temp value
      info['g_adj'][str(i)] = []

    with open(filename, "a") as f: # append
      json.dump(info, f)
      f.write('\n')

In [38]:
!rm ../Colab\ Notebooks/Graph2Seq/data/devmap/*

In [39]:
from pandas.core.internals.construction import dataclasses_to_dicts
data_dir = "../Colab Notebooks/Graph2Seq/data/devmap/"

cdfg_data_dict = {
    'train': train_cdfg_data,
    'dev': valid_cdfg_data,
    'test': test_cdfg_data,
}

programl_data_dict = {
    'train': train_programl_data,
    'dev': valid_programl_data,
    'test': test_programl_data,
}

for data_type in ["cdfg", "programl"]:
  for split in ['train', 'dev', 'test']:
    if data_type == "cdfg":
      filepath = data_dir + 'cdfg_' + split + '.data'
      data_dict = cdfg_data_dict
      generate_padded_input_data(data_dict[split], filepath, MAX_CDFG_NODES, MAX_CDFG_FEAT)
    else:
      filepath = data_dir + 'programl_' + split + '.data' 
      data_dict = programl_data_dict
      generate_padded_input_data(data_dict[split], filepath, MAX_PROGRAML_NODES, MAX_PROGRAML_FEAT)
    print(f"Generated data at {filepath}")

Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/cdfg_train.data
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/cdfg_dev.data
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/cdfg_test.data
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/programl_train.data
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/programl_dev.data
Generated data at ../Colab Notebooks/Graph2Seq/data/devmap/programl_test.data
